<a href="https://colab.research.google.com/github/felladib/H_SentimentAnalysis_REC/blob/main/UpdatesRatings_senti_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip list


In [ ]:
from model_SA import MyLSTM

In [ ]:
!pip uninstall tensorflow
!pip uninstall tensorflow-text
!pip install tensorflow
!pip install tensorflow-text

Found existing installation: tensorflow 2.11.0
Uninstalling tensorflow-2.11.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.9/dist-packages/tensorflow-2.11.0.dist-info/*
    /usr/local/lib/python3.9/dist-packages/tensorflow/*
Proceed (Y/n)? Y
Y
  Successfully uninstalled tensorflow-2.11.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 KB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 KB 8.5 MB/s eta 0:00:00
     ━━━━━━

In [ ]:
#!pip install --upgrade tensorflow_hub
!pip3 install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
import tensorflow as tf
import pandas as pd


In [ ]:
ptr = pd.read_csv('/content/drive/MyDrive/Train.csv')
pte = pd.read_csv('/content/drive/MyDrive/Test.csv')
pv = pd.read_csv('/content/drive/MyDrive/Valid.csv')

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#pd = pd.iloc[:10000, :]
#pt = pt.iloc[:2000, :]
#pv = pv.iloc[:2000, :]

In [ ]:

X_train = ptr['text']
y_train = ptr['label']
X_test = pte['text']
y_test = pte['label']
X_valid = pv['text']
y_valid = pv['label']


In [ ]:
bert_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')


In [ ]:
from keras.engine import training
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text ,training = False)

preprocessing_model = tf.keras.Model(inputs=text_input, outputs=outputs['sequence_output'])


In [ ]:
i = 1
units = 16
l1= 1e-5
l2= 1e-5

if i==1 :
  model_sa = MyLSTM(units , preprocessing_model , l1, l2)

In [ ]:
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 5e-3

In [ ]:
model_sa.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model_sa.fit(X_train, y_train, epochs=EPOCHS, batch_size= BATCH_SIZE, validation_data=(X_valid, y_valid))

# **Somme pondéré**

In [ ]:
#enregistrer l'output de l'analyse de sentiment et l'ajouter comme une colonne dans le dataset
output_vector = model_SA.predict(X).flatten()
dataset['sent'] = output_vector

In [ ]:
#Fonction de calcul de la somme pondéré
def calculer_somme_ponderer(dataset,alpha):
  sp = np.zeros(len(dataset)) #initialiser le vecteur sp avec des 0

  for index, row in dataset.iterrows(): #parourir le dataset
    sp[index] = alpha*row['sent'] + (1-alpha)*row['rating'] #calculer la somme pondéré

  NHF_dataset = dataset.copy() #créer un nouveau dataset pour aléger le dataset et ne pas perdre de données
  NHF_dataset.drop(['text','rating'], axis=1, inplace=True) #supprimer les colonne inutiles pour le NHF
  NHF_dataset['updated_rating'] = sp #Ajouter la colonne avec les ratings mis à jour

  return NHF_dataset #retourner le nouveau dataset

In [ ]:
import pickle

In [ ]:
NHF_dataset = calculer_somme_ponderer(dataset,0.8)

#Sauvegarder le fichier pour l'utiliser dans le NHF module
with  open("NHF_dataset.pkl","wb") as file:
pickle.dump(NHF_dataset,file,pickle.HIGHEST_PROTOCOL)

In [ ]:
NHF_dataset

In [ ]:
#ouvrir le fichier enregisté pour vérifier
with  open('/content/NHF_data.pkl',"rb") as file:
    NHF=pickle.load(file)

In [ ]:
NHF

# *Test de la partie calcul de la somme pondéré*
1- Reste à régler les ratings qui sont entre 0 et 5 et le score de sentiment qui entre 0 et 1

In [ ]:
import pandas as pd
import random
import pickle
import numpy as np

In [ ]:
dataset=pd.read_csv(r'/content/drive/MyDrive/Train.csv')

In [ ]:
#créer des ratings random dans le dataset just pour tester

my_arr =[random.randint(0, 5) for i in range(40000)]
dataset['rating'] = my_arr

In [ ]:
#prendre les 5 première valeur pour que se soit rapide
dataset = dataset.iloc[:5, :]

In [ ]:
#0 represents a negative review / 1 positive review
dataset

,text,label,rating
0,I grew up (b. 1965) watching and loving the Th...,0,2
1,"When I put this movie in my DVD player, and sa...",0,3
2,Why do people who do not know what a particula...,0,4
3,Even though I have great interest in Biblical ...,0,2
4,Im a die hard Dads Army fan and nothing will e...,1,0


In [ ]:
def calculer_moyenne_ponderer(dataset,alpha=0):
  moy = np.zeros(len(dataset))
  for index, row in dataset.iterrows():
    moy[index] = alpha*row['label'] + (1-alpha)*row['rating']
    print(alpha)
    print(row['label'])
    print(row['rating'])
    print(moy)

  NHF_dataset = dataset.copy()
  NHF_dataset.drop(['text','rating'], axis=1, inplace=True)
  NHF_dataset['updated_rating'] = moy


  return NHF_dataset

In [ ]:
NHF_data = calculer_moyenne_ponderer(dataset,0.8)

0.8
0
2
[0.4 0.  0.  0.  0. ]
0.8
0
3
[0.4 0.6 0.  0.  0. ]
0.8
0
4
[0.4 0.6 0.8 0.  0. ]
0.8
0
2
[0.4 0.6 0.8 0.4 0. ]
0.8
1
0
[0.4 0.6 0.8 0.4 0.8]


In [ ]:
NHF_data

,label,updated_rating
0,0,0.4
1,0,0.6
2,0,0.8
3,0,0.4
4,1,0.8


In [ ]:
with  open("NHF_data.pkl","wb") as file:
  pickle.dump(NHF_data,file,pickle.HIGHEST_PROTOCOL)

In [ ]:
NHF_data

,label,updated_rating
0,0,0.4
1,0,0.6
2,0,0.8
3,0,0.4
4,1,0.8


In [ ]:
with  open('/content/NHF_data.pkl',"rb") as file:
    NHF=pickle.load(file)

In [ ]:
NHF

,label,updated_rating
0,0,0.4
1,0,0.6
2,0,0.8
3,0,0.4
4,1,0.8
